In [13]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

In [14]:
start_date = "01/31/1957"
end_date = "12/31/2021"

In [29]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
load_dotenv()

connection_string = (
  "postgresql+psycopg2://"
 f"{'USERNAME'}:{'PASSWORD'}"
  "@wrds-pgdata.wharton.upenn.edu:9737/wrds"
)

wrds = create_engine(connection_string, pool_pre_ping=True)

In [30]:
crsp_monthly_query = (
  "SELECT msf.permno, date_trunc('month', msf.mthcaldt)::date AS date, "
         "msf.mthret AS ret, msf.shrout, msf.mthprc AS altprc, "
         "ssih.primaryexch, ssih.siccd "
    "FROM crsp.msf_v2 AS msf "
    "INNER JOIN crsp.stksecurityinfohist AS ssih "
    "ON msf.permno = ssih.permno AND "
       "ssih.secinfostartdt <= msf.mthcaldt AND "
       "msf.mthcaldt <= ssih.secinfoenddt "
   f"WHERE msf.mthcaldt BETWEEN '{start_date}' AND '{end_date}' "
          "AND ssih.sharetype = 'NS' "
          "AND ssih.securitytype = 'EQTY' "  
          "AND ssih.securitysubtype = 'COM' " 
          "AND ssih.usincflg = 'Y' " 
          "AND ssih.issuertype in ('ACOR', 'CORP') " 
          "AND ssih.primaryexch in ('N', 'A', 'Q') "
          "AND ssih.conditionaltype in ('RW', 'NW') "
          "AND ssih.tradingstatusflg = 'A'"
)

crsp_monthly = (pd.read_sql_query(
    sql = crsp_monthly_query,
    con = wrds,
    dtype = {"permno": int, "siccd": int},
    parse_dates = {"date"})
  .assign(shrout = lambda x: x["shrout"]*1000)
)

In [38]:
os.chdir('/Users/eddiewu/Downloads/gu_kelly_xiu/')

if not os.path.exists("data"):
  os.makedirs("data")
    
tidy_finance = sqlite3.connect(database="data/tidy_finance_python.sqlite")


In [42]:
import pandas_datareader as pdr

factors_ff3_monthly_raw = pdr.DataReader(
  name = "F-F_Research_Data_Factors",
  data_source = "famafrench", 
  start = start_date, 
  end = end_date)[0]

factors_ff3_monthly = (factors_ff3_monthly_raw
  .divide(100)
  .reset_index(names = "date")
  .assign(date = lambda x: pd.to_datetime(x["date"].astype(str)))
  .rename(str.lower, axis = "columns")
  .rename(columns = {"mkt-rf": "mkt_excess"})
)


/Users/eddiewu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/4n/71vyv5y911sc9sp_7rhzkq1r0000gn/T/ipykernel_27186/3406512594.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factors_ff3_monthly_raw = pdr.DataReader(
/var/folders/4n/71vyv5y911sc9sp_7rhzkq1r0000gn/T/ipykernel_27186/3406512594.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factors_ff3_monthly_raw = pdr.DataReader(


In [44]:
(factors_ff3_monthly
  .to_sql(name = "factors_ff3_monthly", 
          con = tidy_finance, 
          if_exists = "replace",
          index = False)
)

780

In [45]:
tidy_finance = sqlite3.connect(database="data/tidy_finance_python.sqlite")


In [46]:

factors_ff3_monthly = pd.read_sql_query(
  sql = "SELECT date, rf FROM factors_ff3_monthly",
  con = tidy_finance,
  parse_dates = {"date"}
)
  
crsp_monthly = (crsp_monthly
  .merge(factors_ff3_monthly, how = "left", on = "date")
  .assign(ret_excess = lambda x: x["ret"]-x["rf"])
  .assign(ret_excess = lambda x: x["ret_excess"].clip(lower=-1))
  .drop(columns = ["rf"])
)

In [50]:
(crsp_monthly
  .to_sql(name = "crsp_monthly", 
          con = tidy_finance, 
          if_exists = "replace",
          index = False)
)

3354914